# Batch Informed Trees algorithm

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from queue import PriorityQueue
import networkx as nx

In [16]:
occ_grid = np.ones((5,5))
occ_grid

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [20]:
def g_t(x):
    raise NotImplementedError

def h_t(x):
    raise NotImplementedError

def h_hat_t(x):
    raise NotImplementedError

def g_hat_t(x):
    raise NotImplementedError

def f_t(x):
    raise NotImplementedError

def prune(t, unconn, c):
    raise NotImplementedError

def sample():
    raise NotImplementedError

def Parent(node):
    raise NotImplementedError

def c(node1, node2):
    raise NotImplementedError

def c_hat(node1, node2):
    raise NotImplementedError

def expand_next_vert(vert, edge, cost):
    raise NotImplementedError

In [ ]:
class Tree(nx.digraph):
    def __init__(self) -> None:
        super().__init__()
    
    def unconnected(self):
        return [self.nodes() for node in self.nodes() if self.out_degree(node) == 0]
    
    def vertices(self):
        return self.nodes()
    
    def edges(self):
        return self.edges()
    
    def f_hat(self, node):
        return self.g_hat(node) + self.h_hat(node)
    
    def g_hat(self, node):
        # return l2 norm of node between node and root
        return np.linalg.norm(np.array(node) - np.array(self.start))
        
    def h_hat(self, node):
        return np.linalg.norm(np.array(node) - np.array(self.goal))
    
    def c_hat(self, node1, node2):
        return np.linalg.norm(np.array(node1) - np.array(node2))
    
    def prune(self, cur_cost):
        x_reuse = []
        x_uncon = self.unconnected()
        for x in self.x_unconn:
            if(self.f_hat(x) >= cur_cost):
                self.remove_node(x)
                
        vertices_sorted = sorted(self.nodes(), key = lambda x: self.g_T(x))
        for node in vertices_sorted:
                        
            if(self.f_hat(node) > cur_cost or self.g_T(node) + self.h_hat(node) > cur_cost):
                self.remove_node(node)
                self.v_soln.remove(node)
                self.v_unexp.remove(node)
                v_parent = self.Parent(node)
                self.remove_edge(v_parent, node)
                
                if self.f_hat(node) <  cur_cost:
                    x_reuse.append(node)
        
        return x_reuse                

    def ExpandNextVertex(self):
        
        v_min = self.Qv.get()
        
        if v_min in self.v_unexp:
            x_near = self.NearestVertex(X_unconn, v_min, r_bit)
        else:
            x_input = [x for x in X_new if x in X_unconn]
            x_near = self.NearestVertex(x_input, v_min, r_bit)
            
        
        for x in x_near:
            if self.g_hat(v_min) + self.c_hat(v_min, x) + self.h_hat(x) < cur_cost:
                self.Qe.put((self.c_hat(v_min, x), v_min, x))
                
        if v_min in self.v_unexp:
            v_near = self.NearestVertex(self.vertices(), v_min, r_bit)
            
            for v in v_near:

             
        self.Qe.put((self.c_hat(v_min, x_near), v_min, x_near))
                
        # raise NotImplementedError

In [22]:
start = (0,0)
goal = (4,4)
tree = nx.Graph()
verts = []
verts.append(start)
tree.add_node(start)
edges = []

x_uconn = [goal]
qv = PriorityQueue()
qe = PriorityQueue()

qv.put((0, start))
vert_unexp = []
vert_unexp.append(start)
x_new = []
x_new.append(goal)

vert_soln = []

cost_i = np.infty

unchanged = 0

while True:
    if qe.empty() and qv.empty():
        x_reuse = prune(tree, x_uconn, cost_i)
        x_sample = sample()
        x_new = []
        x_new.extend(x_reuse)
        x_new.extend(x_sample)
        x_uconn.extend([*set(x_new)])
        list(map(qv.put, [(f_t(x), x) for x in verts]))
    
    while qv.get() <= qe.get():
        expand_next_vert(qv, qe, cost_i)

    v_min, x_min = qe.get()
    
    if g_t(v_min) + c_hat(v_min, x_min) +  h_hat_t(x_min) < cost_i:
        if g_t(v_min) + c_hat(v_min, x_min) < g_t(x_min):
            c_edge = c(v_min, x_min)
            if g_t(v_min) + c_edge +  h_hat_t(x_min) < cost_i:
                if g_t(v_min) + c_edge < g_t(x_min):  
                    unchanged = 0
                    if x_min in verts:
                        v_parent = Parent(x_min)
                        edges.pop((v_min, x_min))
                    else:
                        x_uconn.pop(x_min)
                        verts.append(x_min)
                        qv.put((f_t(x_min), x_min))
                        vert_unexp.append(x_min)

                        if x_min == goal:
                            vert_soln.append(x_min)
                    edges.append((v_min, x_min))
                    cost_i = min(g_t(goal)) if goal in vert_soln else cost_i
    else:
        qv, qe = PriorityQueue(), PriorityQueue()
        unchanged += 1
        
    if unchanged == 5:
        break


KeyboardInterrupt: 